Deep Learning Models -- A collection of various deep learning architectures, models, and tips for TensorFlow and PyTorch in Jupyter Notebooks.
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.6.8
IPython 7.2.0

torch 1.1.0


- Runs on CPU or GPU (if available)

# Model Zoo -- Ordinal Regression CNN -- Beckham and Pal 2016

Implementation of a method for ordinal regression by Beckham and Pal [1] applied to predicting age from face images in the AFAD [2] (Asian Face) dataset using a simple ResNet-34 [3] convolutional network architecture.

Note that in order to reduce training time, only a subset of AFAD (AFAD-Lite) is being used.

- [1] Beckham, Christopher, and Christopher Pal. "[A simple squared-error reformulation for ordinal classification](https://arxiv.org/abs/1612.00775)." arXiv preprint arXiv:1612.00775 (2016).
- [2] Niu, Zhenxing, Mo Zhou, Le Wang, Xinbo Gao, and Gang Hua. "[Ordinal regression with multiple output cnn for age estimation](https://ieeexplore.ieee.org/document/7780901/)." In Proceedings of the IEEE conference on computer vision and pattern recognition, pp. 4920-4928. 2016.
- [3] He, Kaiming, Xiangyu Zhang, Shaoqing Ren, and Jian Sun. "[Deep residual learning for image recognition](http://openaccess.thecvf.com/content_cvpr_2016/html/He_Deep_Residual_Learning_CVPR_2016_paper.html)." In Proceedings of the IEEE conference on computer vision and pattern recognition, pp. 770-778. 2016.




## Imports

In [2]:
import time
import numpy as np
import pandas as pd
import os

import torch.nn as nn
import torch.nn.functional as F
import torch

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image



if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Downloading the Dataset

In [3]:
!git clone https://github.com/afad-dataset/tarball-lite.git

Cloning into 'tarball-lite'...
remote: Enumerating objects: 37, done.
remote: Total 37 (delta 0), reused 0 (delta 0), pack-reused 37
Unpacking objects: 100% (37/37), done.
Checking out files: 100% (30/30), done.


In [4]:
!cat tarball-lite/AFAD-Lite.tar.xz* > tarball-lite/AFAD-Lite.tar.xz

In [5]:
!tar xf tarball-lite/AFAD-Lite.tar.xz

In [6]:
rootDir = 'AFAD-Lite'

files = [os.path.relpath(os.path.join(dirpath, file), rootDir)
         for (dirpath, dirnames, filenames) in os.walk(rootDir) 
         for file in filenames if file.endswith('.jpg')]

In [7]:
len(files)

59344

In [8]:
d = {}

d['age'] = []
d['gender'] = []
d['file'] = []
d['path'] = []

for f in files:
    age, gender, fname = f.split('/')
    if gender == '111':
        gender = 'male'
    else:
        gender = 'female'
        
    d['age'].append(age)
    d['gender'].append(gender)
    d['file'].append(fname)
    d['path'].append(f)

In [9]:
df = pd.DataFrame.from_dict(d)
df.head()

,age,gender,file,path
0,39,female,474596-0.jpg,39/112/474596-0.jpg
1,39,female,397477-0.jpg,39/112/397477-0.jpg
2,39,female,576466-0.jpg,39/112/576466-0.jpg
3,39,female,399405-0.jpg,39/112/399405-0.jpg
4,39,female,410524-0.jpg,39/112/410524-0.jpg


In [10]:
df['age'].min()

'18'

In [11]:
df['age'] = df['age'].values.astype(int) - 18

In [12]:
np.random.seed(123)
msk = np.random.rand(len(df)) < 0.8
df_train = df[msk]
df_test = df[~msk]

In [13]:
df_train.set_index('file', inplace=True)
df_train.to_csv('training_set_lite.csv')

In [14]:
df_test.set_index('file', inplace=True)
df_test.to_csv('test_set_lite.csv')

In [15]:
num_ages = np.unique(df['age'].values).shape[0]
print(num_ages)

22


## Settings

In [16]:
##########################
### SETTINGS
##########################

# Device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

NUM_WORKERS = 4

NUM_CLASSES = 22
BATCH_SIZE = 512
NUM_EPOCHS = 150
LEARNING_RATE = 0.0005
RANDOM_SEED = 123
GRAYSCALE = False

TRAIN_CSV_PATH = 'training_set_lite.csv'
TEST_CSV_PATH = 'test_set_lite.csv'
IMAGE_PATH = 'AFAD-Lite'

## Dataset Loaders

In [17]:
class AFADDatasetAge(Dataset):
    """Custom Dataset for loading AFAD face images"""

    def __init__(self, csv_path, img_dir, transform=None):

        df = pd.read_csv(csv_path, index_col=0)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_paths = df['path']
        self.y = df['age'].values
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.img_paths[index]))

        if self.transform is not None:
            img = self.transform(img)

        label = self.y[index]

        return img, label

    def __len__(self):
        return self.y.shape[0]


custom_transform = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.RandomCrop((120, 120)),
                                       transforms.ToTensor()])

train_dataset = AFADDatasetAge(csv_path=TRAIN_CSV_PATH,
                               img_dir=IMAGE_PATH,
                               transform=custom_transform)


custom_transform2 = transforms.Compose([transforms.Resize((128, 128)),
                                        transforms.CenterCrop((120, 120)),
                                        transforms.ToTensor()])

test_dataset = AFADDatasetAge(csv_path=TEST_CSV_PATH,
                              img_dir=IMAGE_PATH,
                              transform=custom_transform2)


train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=NUM_WORKERS)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=NUM_WORKERS)

## Model

In [18]:
##########################
# MODEL
##########################


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes, grayscale):
        self.num_classes = num_classes
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1, padding=2)
        self.fc = nn.Linear(2048 * block.expansion, num_classes)
        self.a = torch.nn.Parameter(torch.zeros(
            self.num_classes).float().normal_(0.0, 0.1).view(-1, 1))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        logits = self.fc(x)
        probas = torch.softmax(logits, dim=1)
        predictions = ((self.num_classes-1)
                       * torch.sigmoid(probas.mm(self.a).view(-1)))
        return logits, probas, predictions


def resnet34(num_classes, grayscale):
    """Constructs a ResNet-34 model."""
    model = ResNet(block=BasicBlock,
                   layers=[3, 4, 6, 3],
                   num_classes=num_classes,
                   grayscale=grayscale)
    return model

In [19]:
###########################################
# Initialize Cost, Model, and Optimizer
###########################################

def cost_fn(targets, predictions):
    return torch.mean((targets.float() - predictions)**2)


torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
model = resnet34(NUM_CLASSES, GRAYSCALE)

model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## Training

In [20]:
def compute_mae_and_mse(model, data_loader):
    mae, mse, num_examples = torch.tensor([0.]), torch.tensor([0.]), 0
    for features, targets in data_loader:
        features = features.to(DEVICE)
        targets = targets.float().to(DEVICE)
        logits, probas, predictions = model(features)
        assert len(targets.size()) == 1
        assert len(predictions.size()) == 1
        predicted_labels = torch.round(predictions).float()
        num_examples += targets.size(0)
        mae += torch.abs(predicted_labels - targets).sum()
        mse += torch.sum((predicted_labels - targets)**2)
    mae = mae / num_examples
    mse = mse / num_examples
    return mae, mse


start_time = time.time()
for epoch in range(NUM_EPOCHS):

    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):

        features = features.to(DEVICE)
        targets = targets.to(DEVICE)

        # FORWARD AND BACK PROP
        logits, probas, predictions = model(features)
        assert len(targets.size()) == 1
        assert len(predictions.size()) == 1
        cost = cost_fn(targets, predictions)
        optimizer.zero_grad()

        cost.backward()

        # UPDATE MODEL PARAMETERS
        optimizer.step()

        # LOGGING
        if not batch_idx % 150:
            s = ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f'
                 % (epoch+1, NUM_EPOCHS, batch_idx,
                     len(train_dataset)//BATCH_SIZE, cost))
            print(s)

    s = 'Time elapsed: %.2f min' % ((time.time() - start_time)/60)
    print(s)

Epoch: 001/150 | Batch 0000/0092 | Cost: 42.0424
Time elapsed: 0.92 min
Epoch: 002/150 | Batch 0000/0092 | Cost: 41.3301
Time elapsed: 1.85 min
Epoch: 003/150 | Batch 0000/0092 | Cost: 40.5070
Time elapsed: 2.78 min
Epoch: 004/150 | Batch 0000/0092 | Cost: 40.4149
Time elapsed: 3.72 min
Epoch: 005/150 | Batch 0000/0092 | Cost: 38.0820
Time elapsed: 4.67 min
Epoch: 006/150 | Batch 0000/0092 | Cost: 38.6630
Time elapsed: 5.61 min
Epoch: 007/150 | Batch 0000/0092 | Cost: 36.5432
Time elapsed: 6.54 min
Epoch: 008/150 | Batch 0000/0092 | Cost: 38.1368
Time elapsed: 7.49 min
Epoch: 009/150 | Batch 0000/0092 | Cost: 37.4299
Time elapsed: 8.44 min
Epoch: 010/150 | Batch 0000/0092 | Cost: 32.8457
Time elapsed: 9.38 min
Epoch: 011/150 | Batch 0000/0092 | Cost: 32.5064
Time elapsed: 10.33 min
Epoch: 012/150 | Batch 0000/0092 | Cost: 31.5168
Time elapsed: 11.28 min
Epoch: 013/150 | Batch 0000/0092 | Cost: 29.1672
Time elapsed: 12.23 min
Epoch: 014/150 | Batch 0000/0092 | Cost: 29.7407
Time elapsed

## Evaluation

In [21]:
model.eval()
with torch.set_grad_enabled(False):  # save memory during inference

    train_mae, train_mse = compute_mae_and_mse(model, train_loader)
    test_mae, test_mse = compute_mae_and_mse(model, test_loader)

    s = 'MAE/RMSE: | Train: %.2f/%.2f | Test: %.2f/%.2f' % (
        train_mae, torch.sqrt(train_mse), test_mae, torch.sqrt(test_mse))
    print(s)

s = 'Total Training Time: %.2f min' % ((time.time() - start_time)/60)
print(s)

MAE/RMSE: | Train: 0.90/1.23 | Test: 3.40/4.61
Total Training Time: 141.35 min


In [22]:
%watermark -iv

numpy       1.15.4
pandas      0.23.4
torch       1.1.0
PIL.Image   5.3.0

